In [1]:
from nlp_chat_bot.rag import RAG
from nlp_chat_bot.model.late_chunking_embedding import LateChunkingEmbedding
from langchain_google_genai import ChatGoogleGenerativeAI

G:\CYTech\ING3\nlp_project_chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [2]:
dataset_path = "../data"
model_download_path = "../models"

embedding_function = LateChunkingEmbedding(model_download_path)

llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
rag = RAG(dataset_path, embedding_function, late_chunking=True, llm=llm_gemini)
print("LENGTH", rag.get_num_docs())
docs_retrieved = rag.retrieve(state = {"question": "What is the the article 93 of GRPD?", "context": []})

print("Num docs:", len(docs_retrieved["context"]))

for i in range(len(docs_retrieved["context"])):
    doc = docs_retrieved["context"][i]
    print("\n\n", "#"*30,"\n")
    print(f"doc {i}: (score: {doc.metadata['score']})")
    print(doc.page_content)

G:\CYTech\ING3\nlp_project_chatbot\venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


100%|██████████| 1/1 [00:01<00:00,  1.56s/it]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:01<00:00,  1.64s/it]
0it [00:00, ?it/s]


Embedding and storing 89 documents...


100%|██████████| 89/89 [02:51<00:00,  1.92s/it]

LENGTH 139
Num docs: 3


 ############################## 

doc 0: (score: 36.97541809082031)
Ar ticle 96 
Relationship with previously concluded Ag reements 
International agreements inv olving the transfe r of personal data to third countr ies or internati onal org anisations whic h 
were concluded by Member States pr ior to 24 Ma y 2016, and which compl y with U nion la w as applicable pr ior to that 
dat e, shall remain in f orce until amended, replaced or revok ed. 
Ar ticle 97 
Commission repor ts 
1. By 25 Ma y 2020 and ever y f our ye ars thereaf ter , the Commission shall submit a repor t on the evaluation and 
review of this Regulation to the European P arliament and to the Council. The repor ts shall be made public. 
2. In the context of the evaluations and reviews refer red to in paragraph 1, the Commission shall examine, in 
par ticular , the application and functioning of: 
(a)  Chapt er V on the transfer of personal data to third countr ies or inte r national organisati o

In [ ]:
rag.invoke(query={"question":"What is the the article 93 of GRPD?"})